Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import glob
from pathlib import Path

import ta

import yfinance as yf
import nasdaqdatalink
import pandas_datareader.famafrench as ff
import pandas_datareader.data as web

from keys import nasdaq_api_key

nasdaqdatalink.ApiConfig.api_key = nasdaq_api_key


Market data

In [2]:
sp500 = yf.download(
    tickers="^GSPC",
    start="1980-01-02",
    end="2023-04-01",
    progress=False,
    auto_adjust=True,
)

Technical indicators

In [3]:
def feature_extractor(dframe):
    dframe = dframe.round(2)

    o = dframe["Open"]
    h = dframe["High"]
    l = dframe["Low"]
    c = dframe["Close"]
    v = dframe["Volume"]

    # Momentum

    PPO = ta.momentum.PercentagePriceOscillator(close=c)
    PVO = ta.momentum.PercentageVolumeOscillator(volume=v)
    ROC_10d = ta.momentum.ROCIndicator(close=c, window=10)
    ROC_60d = ta.momentum.ROCIndicator(close=c, window=60)
    RSI = ta.momentum.RSIIndicator(close=c)
    SRSI = ta.momentum.StochRSIIndicator(close=c)
    SO = ta.momentum.StochasticOscillator(high=h, close=c, low=l)
    TSI = ta.momentum.TSIIndicator(close=c)
    UO = ta.momentum.UltimateOscillator(high=h, low=l, close=c)
    WR = ta.momentum.WilliamsRIndicator(high=h, low=l, close=c)

    # Volume

    EMV = ta.volume.EaseOfMovementIndicator(high=h, low=l, volume=v)
    FI = ta.volume.ForceIndexIndicator(close=c, volume=v)
    MFI = ta.volume.MFIIndicator(high=h, low=l, close=c, volume=v)
    NVI = ta.volume.NegativeVolumeIndexIndicator(close=c, volume=v)

    # Volatility

    BB = ta.volatility.BollingerBands(close=c)
    DC = ta.volatility.DonchianChannel(high=h, low=l, close=c)
    KC = ta.volatility.KeltnerChannel(high=h, low=l, close=c)
    UI = ta.volatility.UlcerIndex(close=c)

    # Trend

    AI = ta.trend.AroonIndicator(close=c)
    CCI = ta.trend.CCIIndicator(high=h, low=l, close=c)
    KST = ta.trend.KSTIndicator(close=c)
    MACD = ta.trend.MACD(close=c)
    MI = ta.trend.MassIndex(high=h, low=l)
    STC = ta.trend.STCIndicator(close=c)
    TRIX = ta.trend.TRIXIndicator(close=c)
    VI = ta.trend.VortexIndicator(high=h, low=l, close=c)

    dframe["PPO"] = PPO.ppo_signal()
    dframe["PVO"] = PVO.pvo_signal()
    dframe["MOM10d"] = ROC_10d.roc()
    dframe["MOM60d"] = ROC_60d.roc()
    dframe["RSI"] = RSI.rsi()
    dframe["SRSI"] = SRSI.stochrsi()
    dframe["SO"] = SO.stoch_signal()
    dframe["TSI"] = TSI.tsi()
    dframe["UO"] = UO.ultimate_oscillator()
    dframe["WR"] = WR.williams_r()
    dframe["EMV"] = EMV.sma_ease_of_movement()
    dframe["FI"] = FI.force_index()
    dframe["MFI"] = MFI.money_flow_index()
    dframe["NVI"] = NVI.negative_volume_index()
    dframe["BB"] = BB.bollinger_hband_indicator()
    dframe["BBW"] = BB.bollinger_pband()
    dframe["DC"] = DC.donchian_channel_pband()
    dframe["KC"] = KC.keltner_channel_hband_indicator()
    dframe["UI"] = UI.ulcer_index()
    dframe["AI"] = AI.aroon_indicator()
    dframe["CCI"] = CCI.cci()
    dframe["KST"] = KST.kst_sig()
    dframe["MACD"] = MACD.macd_signal()
    dframe["MI"] = MI.mass_index()
    dframe["STC"] = STC.stc()
    dframe["TRIX"] = TRIX.trix()
    dframe["VI"] = VI.vortex_indicator_diff()

    dframe["VOLA20d"] = dframe["Close"].pct_change().rolling(20).std()
    dframe["VOLA60d"] = dframe["Close"].pct_change().rolling(60).std()
    dframe["CHVOLA"] = dframe["VOLA20d"] - dframe["VOLA60d"]
    dframe["MAXRET"] = dframe["Close"].pct_change().rolling(20).max()
    dframe["MINRET"] = dframe["Close"].pct_change().rolling(20).min()
    dframe["CHMOM"] = dframe["MOM10d"] - dframe["MOM60d"]


    df_ta = dframe.drop(["Open", "High", "Low", "Volume"], axis=1)

    dframe = df_ta.resample("M").last()

    dframe["RETURN"] = dframe["Close"].pct_change()
    dframe.drop("Close", axis=1, inplace=True)

    return dframe


Market ratios

In [4]:
sp500_ratio = nasdaqdatalink.get(
    [
        "MULTPL/SP500_DIV_YIELD_MONTH",
        "MULTPL/SP500_EARNINGS_YIELD_MONTH",
        "MULTPL/SP500_PE_RATIO_MONTH",
    ],
    start_date="1980-01-01",
    end_date="2023-03-31",
)

rule = {
    "MULTPL/SP500_DIV_YIELD_MONTH - Value": "last",
    "MULTPL/SP500_EARNINGS_YIELD_MONTH - Value": "first",
    "MULTPL/SP500_PE_RATIO_MONTH - Value": "first",
}

sp500_ratios = sp500_ratio.resample("M").apply(rule)
sp500_ratios.columns = ["DIVYIELD", "EARNYIELD", "PERATIO"]
sp500_ratios["DIVYIELD"] = sp500_ratios["DIVYIELD"].shift(1)


Rates

In [5]:
fredrates = web.DataReader(
    ["T10Y2Y", "DTB3", "DTB6", "DPRIME", "DFF"],
    "fred",
    start="1980-01-01",
    end="2023-03-31",
)

fredrates = fredrates.resample("M").last()
fredrates["TB6TB3"] = fredrates["DTB6"] - fredrates["DTB3"]
fredrates["TB3"] = fredrates["DTB3"].diff()
fredrates["TB6"] = fredrates["DTB6"].diff()
fredrates["PRIME"] = fredrates["DPRIME"].diff()
fredrates["FF"] = fredrates["DFF"].diff()
fredrates.drop(["DTB3", "DTB6", "DPRIME", "DFF"], axis=1, inplace=True)


Sentiment
https://www.frbsf.org/economic-research/indicators-data/daily-news-sentiment-index/

In [6]:
senti = pd.read_csv(".data/misc/news_sentiment_data.csv", index_col="date")
senti.index = pd.to_datetime(senti.index, format="%d.%m.%Y")
senti.columns = ["SENT1d"]
senti["SENT1d"] = pd.to_numeric(senti["SENT1d"])
senti["SENT20d"] = senti["SENT1d"].rolling(20).sum()
senti = senti.resample("M").last()
senti["SENTMA"] = senti["SENT20d"].rolling(3).mean()


Risk free rate

In [7]:
ff_dict = web.DataReader(
    "F-F_Research_Data_Factors", "famafrench", start="1980-01-02", end="2023-03-31"
)

RF = ff_dict[0].div(100)["RF"]


Macro

In [8]:
fred_real = web.DataReader(
    ["CPIAUCSL", "UNRATE"], "fred", start="1980-01-01", end="2023-03-31"
)

fred_real["CPIAUCSL"] = fred_real["CPIAUCSL"].shift(1)
fred_real["CPIAUCSL"] = fred_real["CPIAUCSL"].pct_change()
fred_real["UNRATE"] = fred_real["UNRATE"].shift(1)
fred_real.columns = ["CPI", "UNRATE"]

fred_real = fred_real.resample("M").last()


Buildung dataframe

In [9]:
macros = sp500_ratios.join(fredrates)
macros = macros.join(senti)
macros = macros.join(fred_real)
macros["RF"] = RF.values


In [10]:
dframe_feature = feature_extractor(sp500)
dframe_feature = dframe_feature.join(macros)
dframe_feature["EXCESS_RETURN"] = dframe_feature["RETURN"] - dframe_feature["RF"]
dframe_feature["AVG_EXCESS_RETURN"] = dframe_feature["EXCESS_RETURN"].rolling(3).mean()
dframe_feature["EXCESS_RETURN_T+1"] = dframe_feature["EXCESS_RETURN"].shift(-1)
dframe_feature.dropna(axis=0, inplace=True)

In [17]:
dframe_feature.to_csv(".data/time_series.csv")
